In [1]:
from mexc_sdk import Spot
import os 
from dotenv import load_dotenv
import pandas as pd
import numpy as np
from datetime import datetime

In [2]:
now = datetime.now().date()

In [3]:
symbol = input("Symbol = ")
folder_save = f"../../data/{symbol}"

start_date = f"{now.year}-01-01"

to_date = str(now)

is_get_now = input("Get now? (y/n)").strip().lower()[0] == "y"

interval = input("""interval = 
                 
                "1m" - "1 minute"
                "5m" - "5 minute"
                "15m" - "15 minute"
                "30m" - "30 minute"
                "60m" - "60 minute"
                "4h" - "4 hour"
                "1d" - "1 day"
                "1W" - "1 week"
                "1M" - "1 month"
                 """).strip()

In [4]:
load_dotenv(dotenv_path="./apikey.txt")

True

In [5]:
access_key = os.getenv("access_key")
secret_key = os.getenv("secret_key")

In [6]:
spot = Spot(api_key=access_key, api_secret=secret_key)

In [7]:
data_from_mexc = spot.klines(symbol.upper() + "USDT", interval=interval, 
    # options={
    #     "startTime": start_date,
    #     "endTime": to_date
    # }
    )

In [8]:
df_origin_mexc = pd.DataFrame(data_from_mexc)

In [9]:
df_origin_mexc.columns

RangeIndex(start=0, stop=8, step=1)

In [10]:
df_origin_mexc["Date"] = pd.to_datetime(df_origin_mexc.iloc[:, 0], unit="ms")

In [11]:
mapping_prices = {1: "Open",
                  2: "High",
                  3: "Low",
                  4: "Close",
                  5: "Volume", }

In [12]:
for index in mapping_prices:
    df_origin_mexc[mapping_prices[index]] = df_origin_mexc.iloc[:, index]

In [13]:
df_from_mexc = df_origin_mexc[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [14]:
now = datetime.now().date()

In [15]:
now

datetime.date(2024, 12, 1)

In [16]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,29800,30061.7,29726.34,29901.71,3547.116708,2023-07-21
1,29901.71,29998.99,29622.24,29794,1956.832041,2023-07-22
2,29794,30350,29730.01,30083.76,2999.126007,2023-07-23
3,30083.76,30099.58,28867,29176.5,5278.560984,2023-07-24
4,29176.5,29373.67,29047.07,29228.91,3256.319756,2023-07-25
...,...,...,...,...,...,...
495,91965.16,97208.21,91792.14,95863.12,9496.18018258,2024-11-27
496,95863.12,96564,94640,95643.97,9872.55541859,2024-11-28
497,95643.97,98619.99,95364.99,97460.01,8612.00588688,2024-11-29
498,97460.01,97463.95,96092.01,96407.99,4571.59970941,2024-11-30


In [17]:
if not is_get_now:
    df_from_mexc = df_from_mexc[df_from_mexc["Date"] != str(now)]

In [18]:
old_df = pd.read_csv(f"{folder_save}/{symbol.lower()}_to_now_{interval.lower()}.csv")
old_df["Date"] = pd.to_datetime(old_df["Date"])

# min_date_in_mexc = str(df_from_mexc["Date"].min())
# old_df = old_df[old_df["Date"] < min_date_in_mexc]



In [19]:
old_df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,2.105680e+07,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,3.448320e+07,2014-09-18
2,424.102997,427.834991,384.532013,394.795990,3.791970e+07,2014-09-19
3,394.673004,423.295990,389.882996,408.903992,3.686360e+07,2014-09-20
4,408.084991,412.425995,393.181000,398.821014,2.658010e+07,2014-09-21
...,...,...,...,...,...,...
3722,97900.010000,98871.800000,92611.860000,93018.970000,7.432218e+03,2024-11-25
3723,93018.970000,94972.400000,90791.130000,91965.160000,6.497888e+03,2024-11-26
3724,91965.160000,97208.210000,91792.140000,95863.120000,9.496180e+03,2024-11-27
3725,95863.120000,96564.000000,94640.000000,95643.970000,9.872555e+03,2024-11-28


In [20]:
old_df = old_df[['Open', 'High',
                'Low', 'Close',     
                'Volume', 'Date']]

In [21]:
old_df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,2.105680e+07,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,3.448320e+07,2014-09-18
2,424.102997,427.834991,384.532013,394.795990,3.791970e+07,2014-09-19
3,394.673004,423.295990,389.882996,408.903992,3.686360e+07,2014-09-20
4,408.084991,412.425995,393.181000,398.821014,2.658010e+07,2014-09-21
...,...,...,...,...,...,...
3722,97900.010000,98871.800000,92611.860000,93018.970000,7.432218e+03,2024-11-25
3723,93018.970000,94972.400000,90791.130000,91965.160000,6.497888e+03,2024-11-26
3724,91965.160000,97208.210000,91792.140000,95863.120000,9.496180e+03,2024-11-27
3725,95863.120000,96564.000000,94640.000000,95643.970000,9.872555e+03,2024-11-28


In [22]:
max_date_in_old_df = str(old_df["Date"].max())

In [23]:
max_date_in_old_df

'2024-11-29 00:00:00'

In [24]:
df_from_mexc

,Open,High,Low,Close,Volume,Date
0,29800,30061.7,29726.34,29901.71,3547.116708,2023-07-21
1,29901.71,29998.99,29622.24,29794,1956.832041,2023-07-22
2,29794,30350,29730.01,30083.76,2999.126007,2023-07-23
3,30083.76,30099.58,28867,29176.5,5278.560984,2023-07-24
4,29176.5,29373.67,29047.07,29228.91,3256.319756,2023-07-25
...,...,...,...,...,...,...
494,93018.97,94972.4,90791.13,91965.16,6497.88820608,2024-11-26
495,91965.16,97208.21,91792.14,95863.12,9496.18018258,2024-11-27
496,95863.12,96564,94640,95643.97,9872.55541859,2024-11-28
497,95643.97,98619.99,95364.99,97460.01,8612.00588688,2024-11-29


In [25]:
df_today = df_from_mexc[df_from_mexc["Date"] > max_date_in_old_df]


In [26]:
df_today

,Open,High,Low,Close,Volume,Date
498,97460.01,97463.95,96092.01,96407.99,4571.59970941,2024-11-30


In [27]:
old_df["Date"] = old_df["Date"].dt.date
df_today["Date"] = df_today["Date"].dt.date
df = pd.concat([old_df, df_today], ignore_index=True)
df["Date"] = pd.to_datetime(df["Date"])

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_12440\1586841953.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_today["Date"] = df_today["Date"].dt.date


In [28]:
df

,Open,High,Low,Close,Volume,Date
0,465.864014,468.174011,452.421997,457.334015,21056800.0,2014-09-17
1,456.859985,456.859985,413.104004,424.440002,34483200.0,2014-09-18
2,424.102997,427.834991,384.532013,394.79599,37919700.0,2014-09-19
3,394.673004,423.29599,389.882996,408.903992,36863600.0,2014-09-20
4,408.084991,412.425995,393.181,398.821014,26580100.0,2014-09-21
...,...,...,...,...,...,...
3723,93018.97,94972.4,90791.13,91965.16,6497.888206,2024-11-26
3724,91965.16,97208.21,91792.14,95863.12,9496.180183,2024-11-27
3725,95863.12,96564.0,94640.0,95643.97,9872.555419,2024-11-28
3726,95643.97,98619.99,95364.99,97460.01,8612.005887,2024-11-29


In [29]:
symbol = symbol.replace(".", "_").lower()

In [30]:
min_date = df["Date"].min().date()
max_date = df["Date"].max().date()

os.makedirs(folder_save, exist_ok=True)
file_name = f"{folder_save}/{symbol}_to_now_{interval.lower()}.csv"

In [31]:
file_name = file_name.replace("-", "_")

In [32]:
file_name

'../../data/btc/btc_to_now_1d.csv'

In [33]:
df.to_csv(file_name, index=False)